In [5]:
! pip install langchain_community tiktoken langchain-huggingface huggingface_hub langchainhub chromadb langchain ipywidgets --quiet

In [18]:
from huggingface_hub import login
login()

In [8]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace

In [19]:
import getpass
import os

if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your token: ")

Enter your token:  ········


In [24]:
os.environ["USER_AGENT"]="Tanmay"

In [20]:
llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat = ChatHuggingFace(llm=llm, verbose=True)

In [44]:
messages = [
    ("system", "You are a helpful bot. Translate the user sentence to French."),
    ("human", "I love programming.")
]

chat.invoke("I love programming.")

AIMessage(content="I'm glad to hear that you have an affinity for coding! What aspect of programming do you enjoy the most? Is it the logic, creating new applications, solving problems, or something else?", additional_kwargs={}, response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=42, prompt_tokens=7, total_tokens=49), 'model': '', 'finish_reason': 'stop'}, id='run-1f4d6023-f858-4afc-993a-f36883c04e9a-0')

In [33]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate

In [26]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [27]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [28]:
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=HuggingFaceEmbeddings())

In [29]:
retriever = vectorstore.as_retriever()

In [49]:
prompt = PromptTemplate.from_template("You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:")

In [50]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [51]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chat
    | StrOutputParser()
)

In [52]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is the process of breaking down a complex task into smaller sub-tasks for more manageable execution. With HuggingGPT, this can be facilitated by LLM through prompting, task-specific instructions, or human inputs like in the context provided.'